# Análisis Naive Bayes
Autores:
- Gerson Ramírez
- Diego Valenzuela

In [28]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

df = pd.read_csv("./processed_data.csv")

print(df.head())


   Id  MSSubClass  MSZoning  LotFrontage  LotArea  Street  LotShape  \
0   1          60         3         65.0     8450       1         3   
1   2          20         3         80.0     9600       1         3   
2   3          60         3         68.0    11250       1         0   
3   4          70         3         60.0     9550       1         0   
4   5          60         3         84.0    14260       1         0   

   LandContour  Utilities  LotConfig  ...  3SsnPorch  ScreenPorch  PoolArea  \
0            3          0          4  ...          0            0         0   
1            3          0          2  ...          0            0         0   
2            3          0          4  ...          0            0         0   
3            3          0          0  ...          0            0         0   
4            3          0          2  ...          0            0         0   

   MiscVal  MoSold  YrSold  SaleType  SaleCondition  SalePrice  PriceCategory  
0        0       2

## Identificación de valores nulos y tipos de datos

Se analiza la presencia de valores nulos en el dataset y se identifican los tipos de datos de cada columna. Esto permitirá definir estrategias de limpieza y preprocesamiento antes de entrenar el modelo.

In [29]:
df.info()
df.isnull().sum().sort_values(ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   int64  
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   int64  
 6   LotShape       1460 non-null   int64  
 7   LandContour    1460 non-null   int64  
 8   Utilities      1460 non-null   int64  
 9   LotConfig      1460 non-null   int64  
 10  LandSlope      1460 non-null   int64  
 11  Neighborhood   1460 non-null   int64  
 12  Condition1     1460 non-null   int64  
 13  Condition2     1460 non-null   int64  
 14  BldgType       1460 non-null   int64  
 15  HouseStyle     1460 non-null   int64  
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

Id               0
HalfBath         0
GarageType       0
Fireplaces       0
Functional       0
                ..
MasVnrArea       0
MasVnrType       0
Exterior2nd      0
Exterior1st      0
PriceCategory    0
Length: 77, dtype: int64

## División del conjunto de datos en entrenamiento y prueba

Se separa el dataset en dos subconjuntos: entrenamiento (80%) y prueba (20%). La variable objetivo es `SalePrice`. Esta división permitirá evaluar el rendimiento del modelo en datos no vistos durante el entrenamiento.


In [30]:

X = df.drop(columns=["SalePrice", "Id"])
y = df["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



## Aplicación de Naive Bayes para regresión

Se entrena un modelo de Naive Bayes para estimar el precio de las viviendas. Dado que Naive Bayes no es un algoritmo de regresión, se adapta usando GaussianNB. Antes del entrenamiento, se escalan los datos para mejorar el desempeño del modelo.

In [31]:

features = ['GrLivArea', 'OverallQual', '1stFlrSF', 'GarageCars', 'YearBuilt', 'LotArea', ]

X = df[features]
y = df["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = GaussianNB()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

In [32]:
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}, RMSE: {rmse}, R2: {r2}")


MSE: 3123767908.839041, RMSE: 55890.678192691856, R2: 0.5927464068070458


## Análisis de los resultados del modelo de Naive Bayes para regresión

Los resultados obtenidos en la evaluación del modelo son:

- **MSE:** 3,384,173,778.46 
- **RMSE:** 5,8173.65
- **R²:** 0.559  

El modelo explica aproximadamente **el 55.9% de la variabilidad** en los precios de las viviendas, pero el error RMSE de **3,384,173,778.46** es relativamente alto, indicando que las predicciones no son precisas. 

Naive Bayes no es un modelo comúnmente utilizado para regresión, y estos resultados reflejan que no captura bien la relación entre las variables predictoras y el precio de las viviendas.

Para determinar si existen modelos más precisos, se procederá a comparar su desempeño con **Regresión Lineal y Árbol de Regresión**.


## Comparación del modelo de Naive Bayes con Árbol de Decisión y Random Forest

Se comparan los resultados obtenidos en la regresión con Naive Bayes frente a otros modelos previamente entrenados:

| Modelo                     | MSE           | RMSE       | R²     |
|----------------------------|--------------|-----------|--------|
| **Naive Bayes**            | 3,384,173,778  | 58,173.65 | 0.5588 |
| **Árbol de Decisión (Prof. 5)** | 1,493,659,000 | 38,647.88 | 0.8053 |
| **Random Forest Regressor** | 813,972,852  | 28,530.21 | 0.8939 |

### **Análisis de los resultados**
- **Naive Bayes mostró el peor desempeño** con el mayor error (MSE y RMSE) y el menor R² (`0.5588`). Esto indica que no logra capturar bien la relación entre las variables predictoras y `SalePrice`.
- **Árbol de Decisión** tuvo un desempeño moderado con un R² de `0.8053`, mejorando significativamente la predicción en comparación con Naive Bayes.
- **Random Forest** obtuvo los mejores resultados con un **MSE y RMSE más bajos**, además de un **R² de 0.8939**, lo que sugiere que captura mejor la variabilidad del precio de las viviendas.

### **Conclusión**
El modelo de **Naive Bayes no es adecuado para regresión en este conjunto de datos**, ya que su precisión es significativamente menor que la de los otros modelos. **Random Forest se posiciona como la mejor opción**, logrando la mayor precisión en la estimación de precios.


## Modelo de Clasificación con Naive Bayes usando PriceCategory

En este paso se entrena un modelo de clasificación con Naive Bayes Gaussiano, utilizando como variable objetivo la columna `PriceCategory`, la cual clasifica las viviendas en tres categorías: Barata, Media y Cara. El objetivo del modelo es predecir correctamente esta clasificación a partir de variables numéricas del dataset.

In [33]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

X = df[features]
y = df["PriceCategory"]

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X, y, test_size=0.2, random_state=42)

scaler_class = StandardScaler()
X_train_class_scaled = scaler_class.fit_transform(X_train_class)
X_test_class_scaled = scaler_class.transform(X_test_class)

model_class = GaussianNB()
model_class.fit(X_train_class_scaled, y_train_class)

y_pred_class = model_class.predict(X_test_class_scaled)


## Evaluación del Modelo de Clasificación con Naive Bayes

Se evalúa el desempeño del modelo de clasificación Naive Bayes utilizando el conjunto de prueba. Para ello, se utilizan métricas comunes como el reporte de clasificación y la matriz de confusión, que permiten identificar el nivel de precisión, recall y F1-score por cada categoría.


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test_class, y_pred_class))
print(classification_report(y_test_class, y_pred_class))
conf_mat = confusion_matrix(y_test_class, y_pred_class)

labels = sorted(df["PriceCategory"].unique())

plt.figure(figsize=(7,6))
sns.heatmap(conf_mat, annot=True, fmt='d', xticklabels=labels, yticklabels=labels, cmap="Blues")
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusión - Clasificación Naive Bayes')
plt.show()

# Reporte de clasificación
report = classification_report(y_test_class, y_pred_class, output_dict=True)
df_report = pd.DataFrame(report).transpose().drop(['accuracy', 'macro avg', 'weighted avg'])

# Gráfica de métricas por clase
df_report[['precision', 'recall', 'f1-score']].plot(kind='bar', figsize=(10,6))
plt.title("Métricas de Clasificación por Categoría")
plt.ylabel("Valor")
plt.xticks(rotation=45)
plt.ylim(0, 1.1)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


[[65  1 28]
 [ 0 98 12]
 [ 4 22 62]]
              precision    recall  f1-score   support

        Cara       0.94      0.69      0.80        94
   Economica       0.81      0.89      0.85       110
  Intermedia       0.61      0.70      0.65        88

    accuracy                           0.77       292
   macro avg       0.79      0.76      0.77       292
weighted avg       0.79      0.77      0.77       292

